In [1]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

In [2]:
HTTP_URL_PATTERN = r'^http[s]*://.+'

In [3]:
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # Create a list to store the hyperlinks
        self.hyperlinks = []

    # Override the HTMLParser's handle_starttag method to get the hyperlinks
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # If the tag is an anchor tag and it has an href attribute, add the href attribute to the list of hyperlinks
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

In [4]:
# Function to get the hyperlinks from a URL
def get_hyperlinks(url):

    # Try to open the URL and read the HTML
    try:
        # Open the URL and read the HTML
        with urllib.request.urlopen(url) as response:

            # If the response is not HTML, return an empty list
            if not response.info().get('Content-Type').startswith("text/html"):
                return []

            # Decode the HTML
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

In [5]:
# Function to get the hyperlinks from a URL that are within the same domain
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return list(set(clean_links))

In [6]:
def crawl(url):
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the text files
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # While the queue is not empty, continue crawling
    while queue:

        # Get the next URL from the queue
        url = queue.pop()
        print(url) # for debugging and to see the progress

        # Save text from the url to a <url>.txt file
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="UTF-8") as f:

            # Get the text from the URL using BeautifulSoup
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # Get the text but remove the tags
            text = soup.get_text()

            # If the crawler gets to a page that requires JavaScript, it will stop the crawl
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")

            # Otherwise, write the text to the file in the text directory
            f.write(text)

        # Get the hyperlinks from the URL and add them to the queue
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)


Mainのコンテンツの取り出し

In [9]:
full_url = "https://www.pmda.go.jp/review-services/drug-reviews/master-files/0007.html"

In [21]:
local_domain = urlparse(full_url).netloc
print(local_domain)

www.pmda.go.jp


In [12]:
soup = BeautifulSoup(requests.get(full_url).content, "html.parser")

In [13]:
print(soup)

<!DOCTYPE html>

<html lang="ja">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width initial-scale=1.0" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<title>原薬等登録原簿（MF） | 独立行政法人 医薬品医療機器総合機構</title>
<meta content="" name="keywords"/>
<meta content="医薬品・医療機器・再生医療等製品の承認審査・安全対策・健康被害救済の3つの業務を行う組織。" name="description"/>
<meta content="原薬等登録原簿（MF）" property="og:title"/>
<meta content="article" property="og:type"/>
<meta content="独立行政法人 医薬品医療機器総合機構" property="og:site_name"/>
<meta content="https://www.pmda.go.jp/review-services/drug-reviews/master-files/0007.html" property="og:url"/>
<meta content="医薬品・医療機器・再生医療等製品の承認審査・安全対策・健康被害救済の3つの業務を行う組織。" property="og:description"/>
<meta content="https://www.pmda.go.jp/files/000253405.png" property="og:image"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="医薬品・医療機器・再生医療等製品の承認審査・安全対策・健康被害救済の3つの業務を行う組織。" name="twitter:description"/>
<

In [14]:
print(soup.find_all('div', ['class' , "inner"]))

[<div class="inner">
<ul class="header__link">
<li class="navskip"><a href="#contents" title="本文へ"><span>本文へ</span></a></li>
<li><a href="/index.html">ホーム</a></li>
<li><a href="/english/index.html" lang="en">English</a></li>
<li><a href="/0007.html">サイトマップ</a></li>
<li><a href="/0006.html">お問い合わせ先</a></li>
</ul>
<ul class="fontsize">
<li class="text">文字サイズ変更</li>
<li class="btn normal"><a href="javascript:textSizeReset();">標準</a></li>
<li class="btn enlarge"><a href="javascript:textSizeUp();">拡大</a></li>
</ul>
</div>, <div class="inner">
<div class="btn__language"><a href="/english/index.html">English</a></div>
<div class="header__title">
<a href="/index.html"><img alt="独立行政法人 医薬品医療機器総合機構" src="/files/000253169.png"/></a>
</div>
<div class="btn__menu">
<div class="inner">
<div class="hamburger_menu">メニュー</div>
<div class="hamburger_close">閉じる</div>
<span></span>
<span></span>
<span></span>
</div><!-- //.inner -->
</div><!-- //.btn__menu -->
</div>, <div class="inner">
<div class="hambu

In [15]:
print(soup.find('main'))

<main class="main">
<div class="link__language"><a href="/english/review-services/reviews/mf/0001.html" lang="en">Click here for English Pages</a></div>
<div class="section">
<div class="inner editor">
<h2>原薬等登録原簿（マスターファイル）（MF）制度について</h2>
<p>　医薬品医療機器法第八十条の六第一項で規定される原薬等登録原簿（以下「MF」という。）制度とは、国内又は外国の原薬等製造業者が原薬等の製造方法・製造管理・品質管理等に係る審査に必要な情報を事前に登録することで、製剤の承認申請者等に対し審査に必要な情報のうち知的財産（ノウハウ）に関わる情報を開示することなく、承認審査に供する制度です。<br>
	　これにより、原薬等製造業者のノウハウが保護されます（但し、公衆衛生上の安全性に関する情報で特に求めのあった場合は除く）。<br>
	　任意登録制ですので、原薬等製造業者がノウハウを保護したい場合に登録を行い、原薬等製造業者がノウハウを開示できる（ノウハウを保護する必要がない）場合は改めて登録する必要はありません。</br></br></p>
<h5>（参考）</h5>
<ul>
<li><a href="/files/000227202.pdf">原薬等登録原簿制度と承認審査</a></li>
</ul>
<p><br>
	　なお、外国の原薬等製造業者がMF登録申請する場合、日本国内に住所を有する当該登録申請等に係る事務を行う者（原薬等国内管理人）を選任し、外国の原薬等製造業者は原薬等国内管理人を通じてMF登録申請する必要があります。また、MF登録申請書、届書その他関係書類は邦文で記載する必要があります。<br>
	　詳細については、平成26年11月17日薬食機参発1117第1号薬食審査発1117第3号「原薬等登録原簿の利用に関する指針について」（以下、MF通知）により、再生医療等製品の形式追加等改めて通知が発出されています。</br></br></p>
<ul>
<li><a href="/files/000203155.pdf">原薬等登録原簿の

HTMLよりメインのコンテンツを得る

In [16]:
print(soup.find('main').get_text())


Click here for English Pages


原薬等登録原簿（マスターファイル）（MF）制度について
　医薬品医療機器法第八十条の六第一項で規定される原薬等登録原簿（以下「MF」という。）制度とは、国内又は外国の原薬等製造業者が原薬等の製造方法・製造管理・品質管理等に係る審査に必要な情報を事前に登録することで、製剤の承認申請者等に対し審査に必要な情報のうち知的財産（ノウハウ）に関わる情報を開示することなく、承認審査に供する制度です。
	　これにより、原薬等製造業者のノウハウが保護されます（但し、公衆衛生上の安全性に関する情報で特に求めのあった場合は除く）。
	　任意登録制ですので、原薬等製造業者がノウハウを保護したい場合に登録を行い、原薬等製造業者がノウハウを開示できる（ノウハウを保護する必要がない）場合は改めて登録する必要はありません。
（参考）

原薬等登録原簿制度と承認審査


	　なお、外国の原薬等製造業者がMF登録申請する場合、日本国内に住所を有する当該登録申請等に係る事務を行う者（原薬等国内管理人）を選任し、外国の原薬等製造業者は原薬等国内管理人を通じてMF登録申請する必要があります。また、MF登録申請書、届書その他関係書類は邦文で記載する必要があります。
	　詳細については、平成26年11月17日薬食機参発1117第1号薬食審査発1117第3号「原薬等登録原簿の利用に関する指針について」（以下、MF通知）により、再生医療等製品の形式追加等改めて通知が発出されています。

原薬等登録原簿の利用に関する指針について

（原薬等登録原簿（MF）制度に関するQ&Aについて）

原薬等登録原簿（MF）制度に関するQ&A
原薬等登録原簿（MF）制度に関するQ&A（その2）
原薬等登録原簿（MF）制度に関するQ&A（その3）
原薬等登録原簿（MF）制度に関するQ&A（その4）

（注） Q&A（その4）の問2は平成30年3月9日付薬生薬審発0309第1号 医薬品審査管理課長、薬生監麻発0309第1号 監視指導・麻薬対策課長連名通知の記の第5により削除
	 
MFの登録対象品目について
　医薬品医療機器法施行規則第280条の2を参照。なお、本項で規定する事項のうち、医療機器に係るものへのMF登録については、登録の適否も含め、まずは審査部門に相談してく

PMDA MFのURLより本文のテキストファイルを作成

In [17]:
url_list = [
"https://www.pmda.go.jp/review-services/drug-reviews/master-files/0007.html",
"https://www.pmda.go.jp/review-services/drug-reviews/master-files/0002.html",
"https://www.pmda.go.jp/review-services/drug-reviews/master-files/0008.html",
"https://www.pmda.go.jp/review-services/drug-reviews/master-files/0005.html",
"https://www.pmda.go.jp/review-services/drug-reviews/master-files/0006.html", ]

In [18]:
local_domain_folder = "pmda_go_jp"

Textフォルダーを作り、さらにローカルドメインのフォルダをつくり、そこにTextとして保管

In [19]:
if not os.path.exists("text/"):
            os.mkdir("text/")

if not os.path.exists("text/"+local_domain_folder+"/"):
    os.mkdir("text/" + local_domain_folder + "/")

for url in url_list:
  with open('text/'+local_domain_folder+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="UTF-8") as f:
    text = BeautifulSoup(requests.get(url).content, "html.parser").find('main').get_text()
    f.write(text)

URLのPDF取り出し

In [22]:
# Function to get the pdf from a URL that are within the same domain
def get_domain_pdfs(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
              #print(link)
              if link.endswith(".pdf"):
                  clean_link = link
                  clean_links.append(clean_link)
               #   print(link, link.endswith(".pdf"))

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link
            #print(clean_link)
            if clean_link.endswith(".pdf"):
              print(clean_link)
              clean_links.append(clean_link)

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            #clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return list(set(clean_links))

In [23]:
url = "https://www.pmda.go.jp/review-services/drug-reviews/master-files/0007.html"
local_domain = "www.pmda.go.jp"

In [ ]:
get_domain_pdfs(local_domain, url)

https://www.pmda.go.jp/files/000160536.pdf
https://www.pmda.go.jp/files/000160131.pdf
https://www.pmda.go.jp/files/000227202.pdf
https://www.pmda.go.jp/files/000203155.pdf
https://www.pmda.go.jp/files/000161338.pdf
https://www.pmda.go.jp/files/000159994.pdf
https://www.pmda.go.jp/files/000161492.pdf
https://www.pmda.go.jp/files/000161666.pdf
https://www.pmda.go.jp/files/000223492.pdf
https://www.pmda.go.jp/files/000229889.pdf
https://www.pmda.go.jp/files/000161782.pdf


['https://www.pmda.go.jp/files/000160131.pdf',
 'https://www.pmda.go.jp/files/000161338.pdf',
 'https://www.pmda.go.jp/files/000161492.pdf',
 'https://www.pmda.go.jp/files/000161782.pdf',
 'https://www.pmda.go.jp/files/000229889.pdf',
 'https://www.pmda.go.jp/files/000223492.pdf',
 'https://www.pmda.go.jp/files/000161666.pdf',
 'https://www.pmda.go.jp/files/000227202.pdf',
 'https://www.pmda.go.jp/files/000159994.pdf',
 'https://www.pmda.go.jp/files/000203155.pdf',
 'https://www.pmda.go.jp/files/000160536.pdf']

PDFのダウンロード

In [ ]:


# ダウンロードしたいPDFのURL
url = 'https://www.pmda.go.jp/files/000161782.pdf'
file_name = url.split('/')[-1]
# URLからPDFを取得する
response = requests.get(url)

# レスポンスが正常かどうかを確認
if response.status_code == 200:
    # PDFファイルの内容をローカルファイルに書き込む
    with open(file_name, 'wb') as f:
        f.write(response.content)


PDFのダウンロード　ファンクション

In [ ]:
def get_pdf_from_url(url):
  file_name = url.split('/')[-1]
# URLからPDFを取得する
  response = requests.get(url)

# レスポンスが正常かどうかを確認
  if response.status_code == 200:
    # PDFファイルの内容をローカルファイルに書き込む
    with open(file_name, 'wb') as f:
        f.write(response.content)

URLリストからPDFのダウンロード　時間がかかる

In [ ]:
local_domain = "www.pmda.go.jp"

In [ ]:
for url in url_list:
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc
    print(local_domain,url)
    # Create a queue to store the URLs to crawl
    #queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the pdf files
    if not os.path.exists("pdf/"):
            os.mkdir("pdf/")

    if not os.path.exists("pdf/"+local_domain+"/"):
            os.mkdir("pdf/" + local_domain + "/")

    if not os.path.exists("pdf/"+local_domain+"/"+url[8:].replace("/", "_")):
            os.mkdir("pdf/" + local_domain + "/"+url[8:].replace("/", "_") )


    # # While the queue is not empty, continue crawling
    #     # Get the next URL from the queue
        #url = queue.pop()
    url_pdf_list = get_domain_pdfs(local_domain, url)
    for url_pdf in url_pdf_list:
      print("url", url_pdf) # for debugging and to see the progress

      # Save text from the url to a <url>.txt file
      with open("pdf/" + local_domain + "/"+url[8:].replace("/", "_") + url_pdf.split('/')[-1], 'wb') as f:
          #open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="UTF-8") as f:

          # URLからPDFを取得する
          response = requests.get(url_pdf)

          # レスポンスが正常かどうかを確認
          if response.status_code == 200:
            # PDFファイルの内容をローカルファイルに書き込む
            #with open(file_name, 'wb') as f:
                f.write(response.content)

      #         seen.add(link)

www.pmda.go.jp
https://www.pmda.go.jp/files/000160536.pdf
https://www.pmda.go.jp/files/000160131.pdf
https://www.pmda.go.jp/files/000227202.pdf
https://www.pmda.go.jp/files/000203155.pdf
https://www.pmda.go.jp/files/000161338.pdf
https://www.pmda.go.jp/files/000159994.pdf
https://www.pmda.go.jp/files/000161492.pdf
https://www.pmda.go.jp/files/000161666.pdf
https://www.pmda.go.jp/files/000223492.pdf
https://www.pmda.go.jp/files/000229889.pdf
https://www.pmda.go.jp/files/000161782.pdf
url https://www.pmda.go.jp/files/000160131.pdf
url https://www.pmda.go.jp/files/000161338.pdf
url https://www.pmda.go.jp/files/000161492.pdf
url https://www.pmda.go.jp/files/000161782.pdf
url https://www.pmda.go.jp/files/000229889.pdf
url https://www.pmda.go.jp/files/000223492.pdf
url https://www.pmda.go.jp/files/000161666.pdf
url https://www.pmda.go.jp/files/000227202.pdf
url https://www.pmda.go.jp/files/000159994.pdf
url https://www.pmda.go.jp/files/000203155.pdf
url https://www.pmda.go.jp/files/000160536

ダウンロードしたPDFをZIPにしてローカルにダウンロード

In [ ]:
!zip -r /content/download.zip /content/pdf


  adding: content/pdf/ (stored 0%)
  adding: content/pdf/www.pmda.go.jp/ (stored 0%)
  adding: content/pdf/www.pmda.go.jp/www.pmda.go.jp_review-services_drug-reviews_master-files_0007.html000223492.pdf (deflated 2%)
  adding: content/pdf/www.pmda.go.jp/www.pmda.go.jp_review-services_drug-reviews_master-files_0006.html000223492.pdf (deflated 2%)
  adding: content/pdf/www.pmda.go.jp/www.pmda.go.jp_review-services_drug-reviews_master-files_0006.html000209172.pdf (deflated 12%)
  adding: content/pdf/www.pmda.go.jp/www.pmda.go.jp_review-services_drug-reviews_master-files_0007.html000203155.pdf (deflated 8%)
  adding: content/pdf/www.pmda.go.jp/www.pmda.go.jp_review-services_drug-reviews_master-files_0006.html000209173.pdf (deflated 11%)
  adding: content/pdf/www.pmda.go.jp/www.pmda.go.jp_review-services_drug-reviews_master-files_0006.html000213480.pdf (deflated 4%)
  adding: content/pdf/www.pmda.go.jp/www.pmda.go.jp_review-services_drug-reviews_master-files_0005.html000197729.pdf (deflated 